In [ ]:
#Dependencies
#!pip install pybliometrics
!pip install pip elsapy
#!pip install python-dotenv

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount= True)
#!ls "/content/drive/My Drive"
import os
from configparser import ConfigParser
config = ConfigParser()
config.read('/content/drive/My Drive/confs/scopus.conf')
os.environ['SCOPUS_KEY'] = config['scopus']['APIKey']


In [ ]:
from pybliometrics.scopus import ScopusSearch
from pybliometrics.scopus.utils import config
config['Authentication']['APIKey'] = '7ee0d918ac44ed6d0cad0e8ddbcf4258'
print(config['Authentication']['APIKey'])

articles = ScopusSearch('ISSN 0169-023X', download=False)
articles.get_results_size()


In [ ]:
global results
global results_df
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import os
import json

client = ElsClient(os.environ['SCOPUS_KEY'])
articles_search = ElsSearch('ISSN(0169-023X) AND PUBYEAR > 2000', 'scopus')
articles_search.execute(client, get_all = False)
results = articles_search.results
results_df  = articles_search.results_df 
print(len(results))
results[0]
#content1  = articles_search.results[0]
#print(content1)
#print(content1['dc:identifier'])
#doc = AbsDoc(content1['dc:identifier'])
#doc.read(client)
#doc.data



#import pandas as pd
#df = pd.DataFrame(articles_search.results)
#df.head()
#print(df.columns)
#for index, row in df.iterrows():
  #print(row)
  #doc = AbsDoc(scp_id = row.eid)
  #doc.read(client)
  #print(row.eid, row['prism:publicationName'],row['prism:doi'], sep=';')
  #print(row['dc:title'], doc.data() if doc.read(client) else  'Not Found',sep='||')
  
  #print(row['dc:title'], doc.data(),sep='||')

In [ ]:
import pandas as pd
#l1 = [ l['@href'] for l in results[0]['link'] if l['@ref']=='scopus']

pd.set_option('max_colwidth', -1)

results_df.head()
#results_df[['dc:identifier','dc:creator','dc:title', 'link', 'prism:doi','prism:issn','prism:volume']]
#results[1]
#results_df.iloc['link']['scopus']

#results_df[['link']]
#with pd.option_context('display.max_rows', None, 'display.max_columns', -1):

#results_df.loc[:,'link']

#results_df.iloc[0].head(1)

#results_df['abstract_link'] = results_df['link']['scopus']
#del results_df['link']


#results_df['scopus_link'] = results_df.loc[results_df['link']['@ref'] == 'scopus', '@href']  #results_df[results_df['link']['@ref'] == ''
#results_df.head()

In [45]:
import lxml.html
import requests
article_url = 'https://www.sciencedirect.com/science/article/abs/pii/S0169023X19304380'
query_abstract = "//article[@role='main']//div[@id='abstracts']"
query_keywords = "//article[@role='main']//div[@class='keywords-section']/div"
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0'
}
response = requests.get(article_url, headers=header)

html = lxml.html.fromstring(response.content.decode('utf-8'))
html.xpath(query_abstract)[0].text_content()
keywords = html.xpath(query_keywords)[0].text_content()
', '.join([t.text_content() for t in html.xpath(query_keywords)])




'Graph query processing, Plan generation, Query evaluation on partitioned graphs, Scalability, Map/Reduce'

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
import json

client = ElsClient(os.environ['SCOPUS_KEY'])
doc = AbsDoc(scp_id = '85079901526')
#doc = AbsDoc(uri = 'https://api.elsevier.com/content/abstract/scopus_id/85079901526')
#doc = FullDoc(doi='10.1016/j.datak.2019.101730')


if doc.read(els_client=client):
   print(doc.data)
else:
   print("Nothing")
print("Request status code: %s" % client.req_status)


{'affiliation': [{'affiliation-city': 'Houston', 'affilname': 'University of Houston', 'affiliation-country': 'United States'}, {'affiliation-city': 'Futuroscope', 'affilname': 'ISAE-ENSMA', 'affiliation-country': 'France'}], 'coredata': {'srctype': 'j', 'eid': '2-s2.0-85079901526', 'prism:coverDate': '2020-03-01', 'prism:aggregationType': 'Journal', 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85079901526', 'subtypeDescription': 'Editorial', 'dc:creator': {'author': [{'ce:given-name': 'Carlos', 'preferred-name': {'ce:given-name': 'Carlos', 'ce:initials': 'C.', 'ce:surname': 'Ordonez', 'ce:indexed-name': 'Ordonez C.'}, '@seq': '1', 'ce:initials': 'C.', '@_fa': 'true', 'affiliation': {'@id': '60005837', '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60005837'}, 'ce:surname': 'Ordonez', '@auid': '57203051107', 'author-url': 'https://api.elsevier.com/content/author/author_id/57203051107', 'ce:indexed-name': 'Ordonez C.'}]}, 'link': [{'@_fa': 'tr